# Depression and Anxiety classification with ERPs

Conceptual replication of the study by [Cavanagh et al. (2019)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6515849/) on own dataset.

Import packages

In [ ]:
import io
import os
import mne
import copy
import glob
import array
import matplotlib
import numpy as np
import pandas as pd
import sklearn.metrics
import seaborn as sns
import scipy.io as sio
import plotly.express as px
import matplotlib.pyplot as plt


from itertools import chain

from scipy.io import loadmat
from scipy import stats

from sklearn import set_config
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import permutation_test_score
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA, FastICA
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score, RepeatedStratifiedKFold
from sklearn.utils import resample

from autoreject import AutoReject

from collections import Counter

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.preprocessing import Xdawn
from mne.decoding import Vectorizer
from mne.decoding import UnsupervisedSpatialFilter

from sklearn.ensemble import BaggingClassifier


# parameters for plotting
plt.rcParams["figure.figsize"] = (10,7)

import seaborn as sns
sns.set_theme(style="whitegrid", palette="deep")

Constatnts

In [ ]:
random_state = 42
signal_frequency = 256

## Load EEG data

In [11]:
data_df = pd.read_pickle('data/sonata_data/sonata_data_GNG_autoreject_freq_short.pkl')

In [ ]:
dep = data_df[(data_df['BDI'] > 13) & (data_df['STAI'] > 41)]
ctrl_dep = data_df[(data_df['BDI'] <= 13) & (data_df['STAI'] > 41)]
anx = data_df[(data_df['BDI'] <= 13) & (data_df['STAI'] > 42)]
ctrl_anx = data_df[(data_df['BDI'] <= 13) & (data_df['STAI'] < 41)]

## Perform classification

In [46]:
# Calculating p-value with permutation test from sci-kit learn
def calculate_p_permutations(estimator, X, y, cv=5, n_permutations=1000, n_jobs=1):

    score_, perm_scores_, pvalue_ = permutation_test_score(
        estimator, X, y, cv=cv, n_permutations=n_permutations, n_jobs=n_jobs
    )

    # summarize
    print(f"     The permutation P-value is = {pvalue_:.4f}")
    print(f"     The permutation score is = {score_:.4f}\n")

    return score_, pvalue_

In [47]:
def evaluate_GS_model(
    pipe,
    X_train, 
    y_train, 
    X_test, 
    y_test, 
    classifier_params,
    pipeline_name,
    cv=StratifiedKFold(n_splits=5),
    predict_test = True,
    predict_train = True,
    ):
    
    # define grid search
    grid_search_model = GridSearchCV(
        pipe,
        classifier_params,
        cv=cv,
        scoring={"roc_auc", "balanced_accuracy", "precision", "recall"},
        refit="balanced_accuracy",
        return_train_score=True,
        verbose=10,
        n_jobs=1,
        
    )

    # fit model
    grid_search_model.fit(X_train, y_train)

    # predict train data
    y_train_pred = grid_search_model.predict(X_train) if predict_train is True else None
    train_score = roc_auc_score(y_train, y_train_pred) if predict_train is True else None 

    # extract mean cv scores
    mean_cv_score = grid_search_model.best_score_
    cv_results_df = pd.DataFrame(grid_search_model.cv_results_).iloc[[grid_search_model.best_index_]]
    cv_splits_scores_df = cv_results_df.filter(regex=r"split\d*_test_roc_auc").reset_index(drop=True) 

    metrics_results_df = cv_results_df.filter(regex=r"mean_test_*").reset_index(drop=True)
    
    # calculate p-value
    cv_p = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores_, pvalue_ = calculate_p_permutations(
            grid_search_model.best_estimator_, X_train, y_train, cv=cv_p
        )

    # save results in dataframe
    this_result = pd.concat(
        [
            pd.DataFrame({
            "model_name": [pipe.steps[-1][0]],
            "pipeline_name": [pipeline_name],
            "train score": [train_score],
            "mean_cv_score": [mean_cv_score],
            "best_model": [grid_search_model.best_estimator_],
            "parameters": [grid_search_model.best_params_],
            "pvalue":[pvalue_],    
            }),
         metrics_results_df,
        ],
    axis=1
    ) 

    return this_result

Define estimators

In [122]:
svc = ('svc' , SVC())
svc_params = dict(
    svc__kernel=["linear"],
    svc__C=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
)

estimators = [
    (svc, svc_params),
]

### Depression vs Control

#### Based on RewP

In [124]:
tmin = 0.20
tmax = 0.30
picks = ['FCz']

In [126]:
rewp_depression_datasets = []

dep_data = [np.mean(mne.filter.resample(epoch['f_good'].average().get_data(tmin=tmin, tmax=tmax, picks=picks), down=1.0)) for epoch in dep['epochs'].to_numpy()]
ctrl_data = [np.mean(mne.filter.resample(epoch['f_good'].average().get_data(tmin=tmin, tmax=tmax,picks=picks), down=1.0)) for epoch in ctrl_dep['epochs'].to_numpy()]

X_rewp = np.array(dep_data)
X_rewp_ctrl = np.array(ctrl_data)

rewp_depression_datasets = [X_rewp, X_rewp_ctrl]

In [127]:
%%capture

vec = Vectorizer()
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)

results_dep_rewp_df = pd.DataFrame()

global_params = {}

X = np.concatenate([rewp_depression_datasets[0], rewp_depression_datasets[1]])
y = np.array(len(rewp_depression_datasets[0]) * [1] +  len(rewp_depression_datasets[1]) * [0])

for (estimator, params) in estimators:
    print(f"Rating {estimator} \n")

    pipeline_name = "RewP_Depression_" + estimator[0]

    clf = Pipeline([estimator])
    classifier_params = {**global_params, **params}

    # enter to grid search
    grid_result = evaluate_GS_model(
        clf,
        X,
        y,
        [],
        [],
        classifier_params = classifier_params,
        pipeline_name = pipeline_name,
        cv=cv,
    )

    print(grid_result)

    results_dep_rewp_df = pd.concat([results_dep_rewp_df, grid_result])

Save the results

In [83]:
results_dep_rewp_df.to_pickle(f"data/sonata_data/results/{task}/depression_rewp_erp_results.pkl")

#### Based on RewP and P3

In [124]:
tmin = 0.20
tmax = 0.30
picks = ['FCz']

tmin_p3 = 0.3
tmax_p3 = 0.4

In [126]:
rewp_depression_datasets = []

dep_data = [np.mean(mne.filter.resample(epoch['f_good'].average().get_data(tmin=tmin, tmax=tmax, picks=picks), down=1.0)) for epoch in dep['epochs'].to_numpy()]
ctrl_data = [np.mean(mne.filter.resample(epoch['f_good'].average().get_data(tmin=tmin, tmax=tmax,picks=picks), down=1.0)) for epoch in ctrl_dep['epochs'].to_numpy()]

dep_p3_data = [np.mean(mne.filter.resample(epoch['f_good'].average().get_data(tmin=tmin_p3, tmax=tmax_p3, picks=picks), down=1.0)) for epoch in dep['epochs'].to_numpy()]
ctrl_p3_data = [np.mean(mne.filter.resample(epoch['f_good'].average().get_data(tmin=tmin_p3, tmax=tmax_p3,picks=picks), down=1.0)) for epoch in ctrl_dep['epochs'].to_numpy()]


X_rewp = np.array(dep_data)
X_rewp_ctrl = np.array(ctrl_data)

X_p3 = np.array(dep_p3_data)
X_p3_ctrl = np.array(ctrl_p3_data)

X_dep = np.column_stack((X_rewp, X_p3))
X_ctr = np.column_stack((X_rewp_ctrl, X_p3_ctrl))

rewp_depression_datasets = [X_dep, X_ctr]

In [127]:
%%capture

vec = Vectorizer()
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)

results_dep_rewp_df = pd.DataFrame()

global_params = {}

X = np.concatenate([rewp_depression_datasets[0], rewp_depression_datasets[1]])
y = np.array(len(rewp_depression_datasets[0]) * [1] +  len(rewp_depression_datasets[1]) * [0])

for (estimator, params) in estimators:
    print(f"Rating {estimator} \n")

    pipeline_name = "RewP_Depression_p3_control_" + estimator[0]

    clf = Pipeline([estimator])
    classifier_params = {**global_params, **params}

    # enter to grid search
    grid_result = evaluate_GS_model(
        clf,
        X,
        y,
        [],
        [],
        classifier_params = classifier_params,
        pipeline_name = pipeline_name,
        cv=cv,
    )

    print(grid_result)

    results_dep_rewp_df = pd.concat([results_dep_rewp_df, grid_result])

Save the results

In [83]:
results_dep_rewp_df.to_pickle(f"data/sonata_data/results/{task}/depression_rewp_erp_p3_results.pkl")

#### Based on FRN

In [67]:
tmin = 0.20
tmax = 0.30
picks = ['FCz']

In [68]:
frn_depression_datasets = []

dep_data = [np.mean(mne.filter.resample(epoch['f_bad'].average().get_data(tmin=tmin, tmax=tmax, picks=picks), down=1.0)) for epoch in dep['epochs'].to_numpy()]
ctrl_data = [np.mean(mne.filter.resample(epoch['f_bad'].average().get_data(tmin=tmin, tmax=tmax,picks=picks), down=1.0)) for epoch in ctrl_dep['epochs'].to_numpy()]

X_rewp = np.array(dep_data)
X_rewp_ctrl = np.array(ctrl_data)

frn_depression_datasets = [X_rewp, X_rewp_ctrl]

In [69]:
%%capture

vec = Vectorizer()
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)

results_dep_frn_df = pd.DataFrame()

global_params = {}

X = np.concatenate([frn_depression_datasets[0], frn_depression_datasets[1]])
y = np.array(len(frn_depression_datasets[0]) * [1] +  len(frn_depression_datasets[1]) * [0])

for (estimator, params) in estimators:
    print(f"Rating {estimator} \n")

    pipeline_name = "FRN_Depression_" + estimator[0]

    clf = Pipeline([estimator])
    classifier_params = {**global_params, **params}

    # enter to grid search
    grid_result = evaluate_GS_model(
        clf,
        X,
        y,
        [],
        [],
        classifier_params = classifier_params,
        pipeline_name = pipeline_name,
        cv=cv,
    )

    print(grid_result)

    results_dep_frn_df = pd.concat([results_dep_frn_df, grid_result])

Save the results

In [82]:
results_dep_frn_df.to_pickle(f"data/sonata_data/results/{task}/depression_frn_erp_results.pkl")

#### Based on FRN and P3

In [67]:
tmin = 0.20
tmax = 0.30
picks = ['FCz']

tmin_p3 = 0.3
tmax_p3 = 0.4

In [68]:
frn_depression_datasets = []

dep_data = [np.mean(mne.filter.resample(epoch['f_bad'].average().get_data(tmin=tmin, tmax=tmax, picks=picks), down=1.0)) for epoch in dep['epochs'].to_numpy()]
ctrl_data = [np.mean(mne.filter.resample(epoch['f_bad'].average().get_data(tmin=tmin, tmax=tmax,picks=picks), down=1.0)) for epoch in ctrl_dep['epochs'].to_numpy()]

dep_p3_data = [np.mean(mne.filter.resample(epoch['f_bad'].average().get_data(tmin=tmin_p3, tmax=tmax_p3, picks=picks), down=1.0)) for epoch in dep['epochs'].to_numpy()]
ctrl_p3_data = [np.mean(mne.filter.resample(epoch['f_bad'].average().get_data(tmin=tmin_p3, tmax=tmax_p3,picks=picks), down=1.0)) for epoch in ctrl_dep['epochs'].to_numpy()]


X_rewp = np.array(dep_data)
X_rewp_ctrl = np.array(ctrl_data)

X_p3 = np.array(dep_p3_data)
X_p3_ctrl = np.array(ctrl_p3_data)

X_dep = np.column_stack((X_rewp, X_p3))
X_ctr = np.column_stack((X_rewp_ctrl, X_p3_ctrl))

frn_depression_datasets = [X_dep, X_ctr]

In [69]:
%%capture

vec = Vectorizer()
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)

results_dep_frn_df = pd.DataFrame()

global_params = {}

X = np.concatenate([frn_depression_datasets[0], frn_depression_datasets[1]])
y = np.array(len(frn_depression_datasets[0]) * [1] +  len(frn_depression_datasets[1]) * [0])

for (estimator, params) in estimators:
    print(f"Rating {estimator} \n")

    pipeline_name = "FRN_Depression_p3_control_" + estimator[0]

    clf = Pipeline([estimator])
    classifier_params = {**global_params, **params}

    # enter to grid search
    grid_result = evaluate_GS_model(
        clf,
        X,
        y,
        [],
        [],
        classifier_params = classifier_params,
        pipeline_name = pipeline_name,
        cv=cv,
    )

    print(grid_result)

    results_dep_frn_df = pd.concat([results_dep_frn_df, grid_result])

Save the results

In [82]:
results_dep_frn_df.to_pickle(f"data/sonata_data/results/{task}/depression_frn_erp_p3_results.pkl")

### Anxiety vs Control

#### Based on RewP

In [71]:
tmin = 0.20
tmax = 0.30
picks = ['FCz']

In [72]:
rewp_anxiety_datasets = []

dep_data = [np.mean(mne.filter.resample(epoch['f_good'].average().get_data(tmin=tmin, tmax=tmax, picks=picks), down=1.0)) for epoch in anx['epochs'].to_numpy()]
ctrl_data = [np.mean(mne.filter.resample(epoch['f_good'].average().get_data(tmin=tmin, tmax=tmax,picks=picks), down=1.0)) for epoch in ctrl_anx['epochs'].to_numpy()]

X_rewp = np.array(dep_data)
X_rewp_ctrl = np.array(ctrl_data)

rewp_anxiety_datasets = [X_rewp, X_rewp_ctrl]

In [74]:
%%capture

vec = Vectorizer()
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)

results_anx_rewp_df = pd.DataFrame()

global_params = {}

X = np.concatenate([rewp_anxiety_datasets[0], rewp_anxiety_datasets[1]])
y = np.array(len(rewp_anxiety_datasets[0]) * [1] +  len(rewp_anxiety_datasets[1]) * [0])

for (estimator, params) in estimators:
    print(f"Rating {estimator} \n")

    pipeline_name = "RewP_Anxiety_" + estimator[0]

    clf = Pipeline([estimator])
    classifier_params = {**global_params, **params}

    # enter to grid search
    grid_result = evaluate_GS_model(
        clf,
        X,
        y,
        [],
        [],
        classifier_params = classifier_params,
        pipeline_name = pipeline_name,
        cv=cv,
    )

    print(grid_result)

    results_anx_rewp_df = pd.concat([results_anx_rewp_df, grid_result])

Save the results

In [81]:
results_anx_rewp_df.to_pickle(f"data/sonata_data/results/{task}/anxiety_rewp_erp_results.pkl")

#### Based on RewP and P3

In [71]:
tmin = 0.20
tmax = 0.30
picks = ['FCz']

tmin_p3 = 0.3
tmax_p3 = 0.4

In [72]:
rewp_anxiety_datasets = []

dep_data = [np.mean(mne.filter.resample(epoch['f_good'].average().get_data(tmin=tmin, tmax=tmax, picks=picks), down=1.0)) for epoch in anx['epochs'].to_numpy()]
ctrl_data = [np.mean(mne.filter.resample(epoch['f_good'].average().get_data(tmin=tmin, tmax=tmax,picks=picks), down=1.0)) for epoch in ctrl_anx['epochs'].to_numpy()]

dep_p3_data = [np.mean(mne.filter.resample(epoch['f_good'].average().get_data(tmin=tmin_p3, tmax=tmax_p3, picks=picks), down=1.0)) for epoch in anx['epochs'].to_numpy()]
ctrl_p3_data = [np.mean(mne.filter.resample(epoch['f_good'].average().get_data(tmin=tmin_p3, tmax=tmax_p3,picks=picks), down=1.0)) for epoch in ctrl_anx['epochs'].to_numpy()]


X_rewp = np.array(dep_data)
X_rewp_ctrl = np.array(ctrl_data)

X_p3 = np.array(dep_p3_data)
X_p3_ctrl = np.array(ctrl_p3_data)

X_dep = np.column_stack((X_rewp, X_p3))
X_ctr = np.column_stack((X_rewp_ctrl, X_p3_ctrl))

rewp_anxiety_datasets = [X_dep, X_ctr]

In [74]:
%%capture

vec = Vectorizer()
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)

results_anx_rewp_df = pd.DataFrame()

global_params = {}

X = np.concatenate([rewp_anxiety_datasets[0], rewp_anxiety_datasets[1]])
y = np.array(len(rewp_anxiety_datasets[0]) * [1] +  len(rewp_anxiety_datasets[1]) * [0])

for (estimator, params) in estimators:
    print(f"Rating {estimator} \n")

    pipeline_name = "RewP_Anxiety_p3_control_" + estimator[0]

    clf = Pipeline([estimator])
    classifier_params = {**global_params, **params}

    # enter to grid search
    grid_result = evaluate_GS_model(
        clf,
        X,
        y,
        [],
        [],
        classifier_params = classifier_params,
        pipeline_name = pipeline_name,
        cv=cv,
    )

    print(grid_result)

    results_anx_rewp_df = pd.concat([results_anx_rewp_df, grid_result])

Save the results

In [81]:
results_anx_rewp_df.to_pickle(f"data/sonata_data/results/{task}/anxiety_rewp_erp_results.pkl")

#### Based on FRN

In [76]:
tmin = 0.20
tmax = 0.30
picks = ['FCz']

In [77]:
frn_anxiety_datasets = []

dep_data = [np.mean(mne.filter.resample(epoch['f_bad'].average().get_data(tmin=tmin, tmax=tmax, picks=picks), down=1.0)) for epoch in anx['epochs'].to_numpy()]
ctrl_data = [np.mean(mne.filter.resample(epoch['f_bad'].average().get_data(tmin=tmin, tmax=tmax,picks=picks), down=1.0)) for epoch in ctrl_anx['epochs'].to_numpy()]

X_rewp = np.array(dep_data)
X_rewp_ctrl = np.array(ctrl_data)

frn_anxiety_datasets = [X_rewp, X_rewp_ctrl]

In [78]:
%%capture

vec = Vectorizer()
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)

results_anx_frn_df = pd.DataFrame()

global_params = {}

X = np.concatenate([frn_anxiety_datasets[0], frn_anxiety_datasets[1]])
y = np.array(len(frn_anxiety_datasets[0]) * [1] +  len(frn_anxiety_datasets[1]) * [0])

for (estimator, params) in estimators:
    print(f"Rating {estimator} \n")

    pipeline_name = "FRN_Depression_p3_control_" + estimator[0]

    clf = Pipeline([estimator])
    classifier_params = {**global_params, **params}

    # enter to grid search
    grid_result = evaluate_GS_model(
        clf,
        X,
        y,
        [],
        [],
        classifier_params = classifier_params,
        pipeline_name = pipeline_name,
        cv=cv,
    )

    print(grid_result)

    results_anx_frn_df = pd.concat([results_anx_frn_df, grid_result])

Save the results

In [80]:
results_anx_frn_df.to_pickle(f"data/sonata_data/results/{task}/anxiety_frn_erp_results.pkl")

#### Based on FRN and P3

In [76]:
tmin = 0.20
tmax = 0.30
picks = ['FCz']

tmin_p3 = 0.3
tmax_p3 = 0.4

In [77]:
frn_anxiety_datasets = []

dep_data = [np.mean(mne.filter.resample(epoch['f_bad'].average().get_data(tmin=tmin, tmax=tmax, picks=picks), down=1.0)) for epoch in anx['epochs'].to_numpy()]
ctrl_data = [np.mean(mne.filter.resample(epoch['f_bad'].average().get_data(tmin=tmin, tmax=tmax,picks=picks), down=1.0)) for epoch in ctrl_anx['epochs'].to_numpy()]

dep_p3_data = [np.mean(mne.filter.resample(epoch['f_bad'].average().get_data(tmin=tmin_p3, tmax=tmax_p3, picks=picks), down=1.0)) for epoch in anx['epochs'].to_numpy()]
ctrl_p3_data = [np.mean(mne.filter.resample(epoch['f_bad'].average().get_data(tmin=tmin_p3, tmax=tmax_p3,picks=picks), down=1.0)) for epoch in ctrl_anx['epochs'].to_numpy()]


X_rewp = np.array(dep_data)
X_rewp_ctrl = np.array(ctrl_data)

X_p3 = np.array(dep_p3_data)
X_p3_ctrl = np.array(ctrl_p3_data)

X_dep = np.column_stack((X_rewp, X_p3))
X_ctr = np.column_stack((X_rewp_ctrl, X_p3_ctrl))

frn_anxiety_datasets = [X_dep, X_ctr]

In [78]:
# %%capture

vec = Vectorizer()
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)

results_anx_frn_df = pd.DataFrame()

global_params = {}

X = np.concatenate([frn_anxiety_datasets[0], frn_anxiety_datasets[1]])
y = np.array(len(frn_anxiety_datasets[0]) * [1] +  len(frn_anxiety_datasets[1]) * [0])

for (estimator, params) in estimators:
    print(f"Rating {estimator} \n")

    pipeline_name = "FRN_Depression_p3_control_" + estimator[0]

    clf = Pipeline([estimator])
    classifier_params = {**global_params, **params}

    # enter to grid search
    grid_result = evaluate_GS_model(
        clf,
        X,
        y,
        [],
        [],
        classifier_params = classifier_params,
        pipeline_name = pipeline_name,
        cv=cv,
    )

    print(grid_result)

    results_anx_frn_df = pd.concat([results_anx_frn_df, grid_result])

Save the results

In [80]:
results_anx_frn_df.to_pickle(f"data/sonata_data/results/{task}/anxiety_frn_erp_p3_results.pkl")